<a href="https://colab.research.google.com/github/abdulhadi240/AdamjeeComputers/blob/main/Reflection_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q poetry langchain langchain-openai black isort langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.

Chains ->


Nodes , edges

In [25]:
%%time
from typing import List, Sequence
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph
from google.colab import userdata

def format_as_markdown(response):
    for message in response:
        # Check if the message is from a human
        if isinstance(message, HumanMessage):
            print(f"**Human Message:**\n\n{message.content}\n")
        # Otherwise, treat it as an AI message (which is still a BaseMessage)
        elif isinstance(message, BaseMessage):
            print(f"**AI Response:**\n\n{message.content}\n")
        else:
            print(f"**Unknown Message Type:**\n\n{message.content}\n")






# Generate the API key from Colab userdata
api_key = userdata.get('OPENAI_API_KEY')

# Ensure API key is retrieved
if api_key is None:
    raise ValueError("API key not found. Please set up the OpenAI API key correctly in Colab.")

generation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a LinkedIn techie influencer assistant tasked with writing excellent and professional LinkedIn posts."),
        ("system", "Generate the best LinkedIn post possible for the user's request."),
        ("system", "Always use a hook for the audience to engage with the post."),
        ("system", "Do **not** use emojis, icons, or any symbols like 🌟, 🚀, or 🤖 in the post. If the post contains any such symbols, remove them immediately."),
        ("system", "If a user provides critique, respond with a revised version of your previous attempts, ensuring no emojis or symbols are used."),
        MessagesPlaceholder(variable_name="messages"),  # Variable name should be "messages"
    ]
)


reflection_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a viral LinkedIn influencer grading a LinkedIn post. Generate a critique and recommendations for the user's post."),
        ("system", "Always provide detailed recommendations, including requests for length, virality, style, etc."),
        ("system", "Do **not** use emojis, icons, or symbols like 🌟, 🚀, or 🤖 in the critique. Remove any such symbols if present."),
        MessagesPlaceholder(variable_name="messages"),  # Variable name should be "messages"
    ]
)

# Create an instance of ChatOpenAI with the API key
llm = ChatOpenAI(api_key=api_key , model_name="gpt-4o")

# Chains
generate_chain = generation_prompt | llm
reflection_chain = reflection_prompt | llm

# Constants
REFLECT = 'reflect'
GENERATE = 'generate'

# Nodes
def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})  # Pass "messages" here

def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflection_chain.invoke({"messages": messages})  # Pass "messages" here
    return [HumanMessage(content=res.content)]

# Graph Building
builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)

# Edge Handling
def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return REFLECT

builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)

# Compile the graph
graph = builder.compile()
print(graph.get_graph().draw_mermaid())

# Test Input
inputs = HumanMessage(content="""
    write a linkedin post for agentic ai .
""")

# Invoke the graph
response = graph.invoke([inputs])
format_as_markdown(response)


%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	reflect(reflect)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	reflect --> generate;
	generate -.-> reflect;
	generate -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

**Human Message:**


    write a linkedin post for agentic ai .


**AI Response:**

Is your organization ready to harness the power of Agentic AI?

In today's rapidly evolving digital landscape, businesses must adapt and innovate to stay ahead. Agentic AI represents the next generation of artificial intelligence, designed to empower organizations with autonomous decision-making capabilities. This groundbreaking technology not only enhances efficiency but also drives strategic growth across various industries.

Imagine a world where AI agents can autonomously perform complex tasks, learn from their environments, and adapt 